# Includes

In [1]:
_exp_name = "sample"

In [2]:
# Import necessary packages.
import numpy as np
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
import datetime
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
import pandas as pd

# This is for the progress bar.
from tqdm.auto import tqdm
import random
import csv

from sklearn.model_selection import KFold
from sklearn import preprocessing

In [3]:
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

# Transforms
Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Please refer to PyTorch official website for details about different transforms.

In [4]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomGrayscale(),
    # transforms.RandomResizedCrop,
    # transforms.AutoAugment(),
    # You may add some transforms here.
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])


# Datasets
The data is labelled by the name, so we load images and label while calling '__getitem__'

In [5]:
class FoodDataset(Dataset):

    def __init__(self, tfm=test_tfm, files = None):
        super(FoodDataset).__init__()
        # self.path = path
        # self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        self.files = files 
        # if files != None:
        #     self.files = files
        # print(f"One sample",self.files[0])
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        try:
            label = int(os.path.split(fname)[1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label

In [6]:
_dataset_dir = "./data/"

def dataset_prefold(path):
    dataset = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
    return dataset

train_set = dataset_prefold(os.path.join(_dataset_dir,"training"))
val_set = dataset_prefold(os.path.join(_dataset_dir,"validation"))
train_val_set = train_set + val_set

le = preprocessing.LabelEncoder()
train_val_encoded = le.fit_transform(train_val_set)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Model

In [7]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# The number of training epochs and patience.
n_epochs = 30
batch_size = 64
patience = 300 # If no improvement in 'patience' epochs, early stop

In [8]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        self.fc = nn.Sequential(
            nn.Linear(512*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 11)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

# Train and Validate

In [9]:
acc_scores = []

with open(f"./log/{_exp_name}_log.csv","a", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["fold", "mode", f"Epoch = {n_epochs}", "Loss", "Acc", "Note"])

In [10]:
for fold, (train_ids, val_ids) in enumerate(kfold.split(train_val_set)):
    # Initialize a model, and put it on the device specified.
    model = Classifier().to(device)

    # For the classification task, we use cross-entropy as the measurement of performance.
    criterion = nn.CrossEntropyLoss()

    # Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)
    
    train_images = [train_val_set[i] for i in train_ids]
    val_images = [train_val_set[i] for i in val_ids]
    
    print(f'FOLD {fold}')
    print(f"One sample of train",train_images[0])
    print(f"One sample of val",val_images[0])
    print('--------------------------------')

    # Initialize trackers, these are not parameters and should not be changed
    stale = 0
    best_acc = 0
    
    # Construct datasets.
    # The argument "loader" tells how torchvision reads the data.
    train_set = FoodDataset(files=train_images, tfm=train_tfm)
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    valid_set = FoodDataset(files=val_images, tfm=test_tfm)
    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    
    for epoch in range(n_epochs):

        # ---------- Training ----------
        # Make sure the model is in train mode before training.
        model.train()

        # These are used to record information in training.
        train_loss = []
        train_accs = []

        for batch in tqdm(train_loader):

            # A batch consists of image data and corresponding labels.
            imgs, labels = batch
            #imgs = imgs.half()
            #print(imgs.shape,labels.shape)

            # Forward the data. (Make sure data and model are on the same device.)
            logits = model(imgs.to(device))

            # Calculate the cross-entropy loss.
            # We don't need to apply softmax before computing cross-entropy as it is done automatically.
            loss = criterion(logits, labels.to(device))

            # Gradients stored in the parameters in the previous step should be cleared out first.
            optimizer.zero_grad()

            # Compute the gradients for parameters.
            loss.backward()

            # Clip the gradient norms for stable training.
            grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

            # Update the parameters with computed gradients.
            optimizer.step()

            # Compute the accuracy for current batch.
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

            # Record the loss and accuracy.
            train_loss.append(loss.item())
            train_accs.append(acc)
            
        train_loss = sum(train_loss) / len(train_loss)
        train_acc = sum(train_accs) / len(train_accs)

        # Print the information.
        print(f"{fold}, [ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
        
        # Update the log.
        with open(f"./log/{_exp_name}_log.csv","a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([fold, "train", f"{epoch+1:03d}", f"{train_loss:.5f}", f"{train_acc:.5f}", ""])

        # ---------- Validation ----------
        # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
        model.eval()

        # These are used to record information in validation.
        valid_loss = []
        valid_accs = []

        # Iterate the validation set by batches.
        for batch in tqdm(valid_loader):

            # A batch consists of image data and corresponding labels.
            imgs, labels = batch
            #imgs = imgs.half()

            # We don't need gradient in validation.
            # Using torch.no_grad() accelerates the forward process.
            with torch.no_grad():
                logits = model(imgs.to(device))

            # We can still compute the loss (but not the gradient).
            loss = criterion(logits, labels.to(device))

            # Compute the accuracy for current batch.
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

            # Record the loss and accuracy.
            valid_loss.append(loss.item())
            valid_accs.append(acc)
            #break

        # The average loss and accuracy for entire validation set is the average of the recorded values.
        valid_loss = sum(valid_loss) / len(valid_loss)
        valid_acc = sum(valid_accs) / len(valid_accs)

        # Print the information.
        print(f"{fold}, [ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

        # update logs
        with open(f"./log/{_exp_name}_log.csv","a", newline="") as f:
            writer = csv.writer(f)
            if valid_acc > best_acc:
                writer.writerow([fold, "val", f"{epoch+1:03d}", f"{valid_loss:.5f}", f"{valid_acc:.5f}", "best"])
            else:
                writer.writerow([fold, "val", f"{epoch+1:03d}", f"{valid_loss:.5f}", f"{valid_acc:.5f}", ""])
        
        
        # save models
        if valid_acc > best_acc:
            print(f"Best model found at epoch {epoch + 1}, saving model")
            torch.save(model.state_dict(), f"{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
            best_acc = valid_acc
            stale = 0
        else:
            stale += 1
            if stale > patience:
                print(f"No improvment {patience} consecutive epochs, early stopping")
                break
                
    acc_scores.append(best_acc)

FOLD 0
One sample of train ./data/training\0_1.jpg
One sample of val ./data/training\0_0.jpg
--------------------------------


  0%|          | 0/167 [00:00<?, ?it/s]

0, [ Train | 001/030 ] loss = 2.14722, acc = 0.23182


  0%|          | 0/42 [00:00<?, ?it/s]

0, [ Valid | 001/030 ] loss = 2.05848, acc = 0.27211
Best model found at epoch 1, saving model


...

1, [ Train | 030/030 ] loss = 0.67489, acc = 0.77320


  0%|          | 0/42 [00:00<?, ?it/s]

1, [ Valid | 030/030 ] loss = 1.27466, acc = 0.62410
FOLD 2
One sample of train ./data/training\0_0.jpg
One sample of val ./data/training\0_121.jpg
--------------------------------


  0%|          | 0/167 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Testing and generate prediction CSV

In [12]:
test_set = dataset_prefold(os.path.join(_dataset_dir,"validation"))
test_set = FoodDataset(files=test_set, tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

In [13]:
model_best = Classifier().to(device)
model_best.load_state_dict(torch.load(f"{_exp_name}_best.ckpt"))
model_best.eval()
prediction = []
with torch.no_grad():
    for data,_ in test_loader:
        test_pred = model_best(data.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        prediction += test_label.squeeze().tolist()

In [15]:
#create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)

current_time = datetime.datetime.now()
month = "%02d" % current_time.month
day = "%02d" % current_time.day
hour = "%02d" % current_time.hour
minute = "%02d" % current_time.minute
time = f"{month}{day}{hour}{minute}"
# result_acc = str(round(best_acc.item(), 2))
result_acc = str(round(0.67827, 2))

df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
df["Category"] = prediction
df.to_csv("./result/submission_" + time + "_" + result_acc + ".csv",index = False)

In [16]:
os.rename("./log/sample_log.csv", "./log/samplelog_" + time + "_" + result_acc + ".csv")